In [ ]:
# Disable some console warnings
import os
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
import sys 
sys.path.append("../training")
import pickle

from save_data import process_data

import hls4ml 
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, BatchNormalization, Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.losses import CategoricalCrossentropy

from tensorflow_model_optimization.python.core.sparsity.keras import prune, pruning_callbacks, pruning_schedule
from tensorflow_model_optimization.sparsity.keras import strip_pruning
import tensorflow_model_optimization as tfmot

from qkeras.qlayers import QDense, QActivation
from qkeras import QBatchNormalization
from qkeras.quantizers import quantized_bits, quantized_relu
from qkeras.utils import _add_supported_quantized_objects
from tensorflow.keras.models import load_model
from qkeras.utils import _add_supported_quantized_objects

# os.environ['PATH'] = os.environ['XILINX_VIVADO'] + '/bin:' + os.environ['PATH']

## Enable/disable training

In [ ]:
TRAIN = False

## Setup data

In [ ]:
def one_hot_encode(data):
    y_encoded = np.zeros([data.shape[0],2], dtype=np.int32)
    for idx, x in enumerate(data):
        if x == 1:
            y_encoded[idx][1] = 1
        else:
            y_encoded[idx][0] = 1
    return y_encoded

In [ ]:
START_WINDOW = 285
END_WINDOW = 385
DATA_DIR = "../data"
MODEL_DIR = "models"

# convert raw ADC data into npy files 
if os.path.exists(f"{DATA_DIR}/X_train_{START_WINDOW}_{END_WINDOW}.npy") == False:
    process_data(
        start_window=START_WINDOW, 
        end_window=END_WINDOW, 
        data_dir=DATA_DIR
    )

In [ ]:
# load data
X_train_val = np.load(os.path.join(DATA_DIR, f'X_train_{START_WINDOW}_{END_WINDOW}.npy'))
X_test = np.load(os.path.join(DATA_DIR, f'X_test_{START_WINDOW}_{END_WINDOW}.npy'))    
y_train_val = np.load(os.path.join(DATA_DIR, f'y_train_{START_WINDOW}_{END_WINDOW}.npy'))
y_test = np.load(os.path.join(DATA_DIR, f'y_test_{START_WINDOW}_{END_WINDOW}.npy'))

y_train_val = one_hot_encode(y_train_val)
y_test = one_hot_encode(y_test)

print("Training:")
print("\tSize:", len(X_train_val))
print("\tSample Shape:", len(X_train_val[0]))
print("\tMean:", X_train_val.mean())
print("\tStd. Dev.:", X_train_val.std())

print("Testing:")
print("\tSize:", len(X_test))
print("\tSample Shape:", len(X_test[0]))
print("\tSample Shape:", X_test.mean())
print("\tStd. Dev.:", X_test.std())

assert len(X_train_val[0]) == (END_WINDOW-START_WINDOW)*2, "ERROR: Specified window does not match loaded dataset shape"
assert len(X_test[0]) == (END_WINDOW-START_WINDOW)*2, "ERROR: Specified window does not match loaded dataset shape"

## Construct a model 
QKeras is "Quantized Keras" for deep heterogeneous quantization of ML models. We're using QDense layer instead of Dense. We're also training with model sparsity, since QKeras layers are prunable.

In [ ]:
INIT_LEARNING_RATE = 1e-2
VALIDATION_SPLIT = 0.3
BATCH_SIZE = 12800
EPOCHS = 50
CHECKPOINT_FILENAME = MODEL_DIR + "/qmodel.h5"
INPUT_SHAPE = (len(X_train_val[0]),)

In [ ]:
if TRAIN:
    input_size = (END_WINDOW-START_WINDOW)*2

    # Define model with quantize layers 
    model = Sequential()
    model.add(QDense(2, input_shape=(input_size,), name='fc1', kernel_quantizer=quantized_bits(6,0,alpha=1), bias_quantizer=quantized_bits(6,0,alpha=1),))
    model.add(BatchNormalization())

    # adding pruning 
    pruning_params = {'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50, final_sparsity=0.80, begin_step=200, end_step=1000)}
    model = prune.prune_low_magnitude(model, **pruning_params)

    print('=============================Model Summary=============================')
    print(model.summary())
    print('=======================================================================')
else:
    print("Training is disabled, load model from file")

In [ ]:
if TRAIN:
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        INIT_LEARNING_RATE,
        decay_steps=100000,
        decay_rate=0.96,
        staircase=True
    )

    callbacks = [
            ModelCheckpoint(
            CHECKPOINT_FILENAME,
            monitor="val_loss",
            verbose=0,
            save_best_only=True,
            save_weights_only=False,
            save_freq="epoch",
        ),
        ReduceLROnPlateau(patience=75, min_delta=1**-6),
        pruning_callbacks.UpdatePruningStep(),
    ]
else:
    print("Training is disabled, load model from file")

## Train the model 

In [ ]:
if TRAIN:
    opt = Adam(learning_rate=INIT_LEARNING_RATE)
    model.compile(
        optimizer=opt, 
        # loss=tf.keras.losses.BinaryCrossentropy(),
        loss=CategoricalCrossentropy(from_logits=True), 
        metrics=['accuracy']
    )

    history = model.fit(
        X_train_val, 
        y_train_val, 
        batch_size=BATCH_SIZE,
        epochs=EPOCHS, 
        validation_split=VALIDATION_SPLIT, 
        shuffle=True, 
        callbacks=callbacks
    )
else:
    print("Training is disabled, load model from file")

Important: Pruning layers must be removed before saving to disk. 

In [ ]:
if TRAIN:
    model = strip_pruning(model)
    model.save(CHECKPOINT_FILENAME)

    history_file = CHECKPOINT_FILENAME.replace(".h5", "-history.pkl")
    with open(history_file, 'wb') as file_pi:
        pickle.dump(history.history, file_pi)

    print(f'Saving history to: {history_file}')
    print(f'Saved checkpoint to: {CHECKPOINT_FILENAME}')
else:
    print("Training is disabled, load model from file")

## Check performance

todo: add ROC curve

In [ ]:
co = {}
_add_supported_quantized_objects(co)
model = load_model(CHECKPOINT_FILENAME, custom_objects=co, compile=False)
y_pred = model.predict(X_test)
print("Keras  Accuracy: {}".format(accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1))))

If you run with the model and data on files, you should expect
```
Keras  Accuracy: 0.8576534653465346
```

## Check sparsity 

In [ ]:
num_layers = len(model.layers)
print(f'Number of layers: {num_layers}')


for idx in range(num_layers):
    w = model.layers[idx].weights[0].numpy()
    h, b = np.histogram(w, bins=100)

    # plot weight distribution
    plt.figure(figsize=(7, 7))
    plt.bar(b[:-1], h, width=b[1] - b[0])
    plt.semilogy()
    plt.savefig(f'model-dist-idx{idx}.png')

    print('% of zeros = {}'.format(np.sum(w == 0) / np.size(w)))

## Build the HLS model 

In [ ]:
sys.path.append("../utils")
from config import print_dict

from tensorflow.keras.models import load_model
from qkeras.utils import _add_supported_quantized_objects

In [ ]:
# Load checkpoint 
co = {}
_add_supported_quantized_objects(co)
model = load_model(CHECKPOINT_FILENAME, custom_objects=co, compile=False)
y_pred = model.predict(X_test)

# Re-evalulate 
y_keras = model.predict(X_test)
print(f'Model acc: {accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_keras, axis=1))}')
print(model.summary()) 

In [ ]:
# Create HLS configuration 
HLSConfig = {}
HLSConfig['Model'] = {}
HLSConfig['Model']['Precision'] = 'ap_fixed<16,6>'  # Default precision
HLSConfig['Model']['ReuseFactor'] = 1  # fully parallelized 

HLSConfig['LayerName'] = {}
for layer in ['fc1_input', 'fc1', 'fc1_linear', 'batch_normalization']:
    HLSConfig['LayerName'][layer] = {}
    HLSConfig['LayerName'][layer]['Precision'] = {}
    HLSConfig['LayerName'][layer]['Trace'] = True

# Input - ZCU216 uses 14-bit ADCS 
HLSConfig['LayerName']['fc1_input']['Precision'] = 'ap_fixed<14,14>' 
# Fc
HLSConfig['LayerName']['fc1']['Precision']['result'] = 'ap_fixed<18,18>'
HLSConfig['LayerName']['fc1']['accum_t'] = 'ap_fixed<18,18>'
# Fc Linear
HLSConfig['LayerName']['fc1_linear']['Precision']['result'] = 'ap_fixed<18,18>'
# Batchnormalization
HLSConfig['LayerName']['batch_normalization']['Precision']['scale'] = 'ap_fixed<20,3>'
HLSConfig['LayerName']['batch_normalization']['Precision']['bias'] = 'ap_fixed<20,3>'
HLSConfig['LayerName']['batch_normalization']['Precision']['result'] = 'ap_fixed<16,6>'

print_dict(HLSConfig)

In [ ]:
OutputDir = 'hls4ml_prj/'
XilinxPart = 'xczu49dr-ffvf1760-2-e'
IOType = 'io_parallel'
ClockPeriod = 3.225  # 3.225ns (307.2 MHz)
HLSFig = OutputDir+'model.png'

hls_model = hls4ml.converters.convert_from_keras_model(
    model=model,
    hls_config=HLSConfig,
    output_dir=OutputDir,
    part=XilinxPart,
    io_type=IOType,
    clock_period=ClockPeriod,
    project_name="NN"
)

print(f"Creating hls4ml project directory {OutputDir}")
hls_model.compile()  # Must compile for C Sim. 

# Visualize model
hls4ml.utils.plot_model(
    hls_model, show_shapes=True, show_precision=True, to_file=HLSFig
)

## Check performance

In [ ]:
# Trace output 
y_hls = hls_model.predict(np.ascontiguousarray(X_test.astype(np.float32))) 
y_hls = np.argmax(y_hls, axis=1)

print(f'Keras Acc: {accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_keras, axis=1))*100:.5}%')
print(f'HLS Acc: {accuracy_score(np.argmax(y_test, axis=1), y_hls)*100:.5}:%')
print(f'CKA: {accuracy_score(np.argmax(y_keras, axis=1), y_hls)*100:.5}%')

## Correlation plots (Keras vs HLS)
Let's compare the output of the Qkeras and HLS model. If properly configured, the HLS activations will be aligned with the Qkeras model. 

In [ ]:
_, hls_trace = hls_model.trace(np.ascontiguousarray(X_test.astype(np.float32))) 
keras_trace = hls4ml.model.profiling.get_ymodel_keras(model, X_test.astype(np.float32)) 

print(f'HLS Keys: {hls_trace.keys()}')
print(f'Keras Keys: {keras_trace.keys()}')

In [ ]:
layers = ['fc1', 'batch_normalization']

for idx, layer in enumerate(layers):
    keras_layer, hls_layer = keras_trace[layer], hls_trace[layer]
    try:
        diff = np.average(np.abs(keras_layer - hls_layer ))
        print(f'{layer}', '\t\t', diff)
        
        plt.figure(figsize=(7, 5))

        plt.scatter(hls_layer.flatten(), keras_layer.flatten())
        min_x = min(keras_layer.min(), hls_layer.min())
        max_x = min(keras_layer.max(), hls_layer.max())

        onnx_min, onnx_max = keras_layer.flatten().min(), keras_layer.flatten().max()
        hls_min, hls_max = hls_layer.flatten().min(), hls_layer.flatten().max()
        
        print(f'hls/keras min: {hls_min}/{onnx_min}')
        print(f'hls/keras max: {hls_max}/{onnx_max}')
        
        plt.plot([min_x, max_x], [min_x, max_x], c='red')
        plt.axhline(min_x, c='red')
        plt.axhline(max_x, c='red')

        plt.title(f'(hls) {layer} -- (keras) {layer}')
        plt.xlabel(f'hls4ml - [{hls_min:.3f},  {hls_max:.3f}]')
        plt.ylabel(f'keras - [{onnx_min:.3f},  {onnx_max:.3f}]')
        plt.yscale('linear')
    except Exception as e:
        print(e)


## Synthesize 

In [ ]:
hls_model.build(
    csim=False,
    synth=True,
    cosim=False,
    export=True,
    vsynth=False,
)

## Check the report

In [ ]:
hls4ml.report.read_vivado_report(f'{OutputDir}/hls4ml_prj')

## Compare QKeras, hls4ml, and board runs

In [ ]:
import csv
import pandas as pd

from ctypes import *
def to_float(i):
    cp = pointer(c_int(i))
    fp = cast(cp, POINTER(c_float))
    return fp.contents.value

### demo00/send_receive_pulse

In [ ]:
# Load CSV files from board runs

# X
df = pd.read_csv(DATA_DIR + '/fpga_testing/X_fpga.csv', header=None)
X_test_board = df.values

# y
df = pd.read_csv(DATA_DIR + '/fpga_testing/y_fpga.csv', header=None)
y_test_board = df.values

In [ ]:
# X -> QKeras -> y
y_qkeras_pred = model.predict(X_test_board)

# X -> HLS -> y (csim)
y_hls_pred = hls_model.predict(np.ascontiguousarray(X_test_board.astype(np.float32)))

In [ ]:
test_size = y_test_board.shape[0]
errors = 0

# Count mismatches (QKeras != HLS)
for i in range(test_size):
    g_ref = to_float(y_test_board[i][0])
    e_ref = to_float(y_test_board[i][1])

    g_hls = y_hls_pred[i][0]
    e_hls = y_hls_pred[i][1]

    mismatch = (g_ref != g_hls) | (e_ref != e_hls)
    print('board[{}] [{:.12f}, {:.12f}]'.format(i, g_ref, e_ref))
    print('hls  [{}] [{:.12f}, {:.12f}] {:s}'.format(i, g_hls, e_hls, '***' if mismatch else ''))
    print('')
    if (mismatch):
        errors = errors + 1

In [ ]:
error_rate = (errors * 100.) / test_size
print('Errors: {:d}/{:d} ({:.2f}%)'.format(errors, test_size, error_rate))

If you run with the model and **board** data on files, you should expect
```
Errors: 761/6235 (12.21%)
```
**ATTENTION:** We should expect _0_ errors (i.e. logits should match) because we are comparing board vs. hls implementations.

Given these and following results, my current guess is a problem with timing/triggering. The NN reads 200 I/Q samples after a trigger event + some delay. The delay in some cases should be a little earlier or later than expected.

```
I, Q, I, Q, I, Q, I, Q, I, Q, I, Q, I, Q, I, Q, I, Q, I, Q, I, Q, I, Q, I, Q, I, Q, I, Q, ..., I, Q, I, Q, I, Q
      ^
      Trigger
      <-------->
         delay
                  <--------------------------------- 200 samples -------------------------------------------->

             <=================================== 200 samples ==========================================>
```

### Mock readout data

#### Ground state

In [ ]:
# Load CSV files from board runs

# X (ground)
df = pd.read_csv(DATA_DIR + '/fpga_testing/malab_g_state_X_fpga.csv', header=None)
X_test_board = df.values

# y (ground)
df = pd.read_csv(DATA_DIR + '/fpga_testing/malab_g_state_y_fpga.csv', header=None)
y_test_board = df.values

In [ ]:
# X -> QKeras -> y
y_qkeras_pred = model.predict(X_test_board)

# X -> HLS -> y (csim)
y_hls_pred = hls_model.predict(np.ascontiguousarray(X_test_board.astype(np.float32)))

In [ ]:
test_size = y_test_board.shape[0]

logit_errors = 0
class_errors = 0

# Count mismatches (QKeras != HLS)
# Count ground predictions (HLS)
for i in range(test_size):
    g_ref = to_float(y_test_board[i][0])
    e_ref = to_float(y_test_board[i][1])

    g_hls = y_hls_pred[i][0]
    e_hls = y_hls_pred[i][1]

    mismatch = (g_ref != g_hls) | (e_ref != e_hls)
    ground_state_ref = (g_ref > e_ref)
    ground_state_hls = (g_hls > e_hls)
    print('board[{}] [{:.12f}, {:.12f}] {:s} '.format(i, g_ref, e_ref, 'G' if ground_state_ref else 'E'))
    print('hls  [{}] [{:.12f}, {:.12f}] {:s} {:s}'.format(i, g_hls, e_hls, 'G' if ground_state_hls else 'E', '***' if mismatch else ''))
    print('')
    if (mismatch):
        logit_errors = logit_errors + 1
    if (not ground_state_ref):
        class_errors = class_errors + 1

In [ ]:
logit_error_rate = (logit_errors * 100.) / test_size
class_error_rate = (class_errors * 100.) / test_size
print('Logit errors: {:d}/{:d} ({:.2f}%)'.format(logit_errors, test_size, logit_error_rate))
print('Class errors: {:d}/{:d} ({:.2f}%)'.format(class_errors, test_size, class_error_rate))

If you run with the model and **board** data on files, you should expect
```
Logit errors: 435/3100 (14.03%)
Class errors: 0/3100 (0.00%)
```
**ATTENTION:** We should expect _0_ errors because we are comparing board vs. hls implementations.

Given these results, my current guess is a problem with timing/triggering. The NN reads 200 I/Q samples after a trigger event + some delay. The delay in some cases should be a little earlier or later than expected.

```
I, Q, I, Q, I, Q, I, Q, I, Q, I, Q, I, Q, I, Q, I, Q, I, Q, I, Q, I, Q, I, Q, I, Q, I, Q, ..., I, Q, I, Q, I, Q
      ^
      Trigger
      <-------->
         delay
                  <--------------------------------- 200 samples -------------------------------------------->

             <=================================== 200 samples ==========================================>
```

#### Excited state

In [ ]:
# Load CSV files from board runs

# X (excited)
df = pd.read_csv(DATA_DIR + '/fpga_testing/malab_e_state_X_fpga.csv', header=None)
X_test_board = df.values

# y (excited)
df = pd.read_csv(DATA_DIR + '/fpga_testing/malab_e_state_y_fpga.csv', header=None)
y_test_board = df.values

In [ ]:
# X -> QKeras -> y
y_qkeras_pred = model.predict(X_test_board)

# X -> HLS -> y (csim)
y_hls_pred = hls_model.predict(np.ascontiguousarray(X_test_board.astype(np.float32)))

In [ ]:
test_size = y_test_board.shape[0]

logit_errors = 0
class_errors = 0

# Count mismatches (QKeras != HLS)
# Count excited predictions (HLS)
for i in range(test_size):
    g_ref = to_float(y_test_board[i][0])
    e_ref = to_float(y_test_board[i][1])

    g_hls = y_hls_pred[i][0]
    e_hls = y_hls_pred[i][1]

    mismatch = (g_ref != g_hls) | (e_ref != e_hls)
    excited_state_ref = (e_ref > g_ref)
    excited_state_hls = (e_hls > g_hls)

    print('board[{}] [{:.12f}, {:.12f}] {:s} '.format(i, g_ref, e_ref, 'E' if excited_state_ref else 'G'))
    print('hls  [{}] [{:.12f}, {:.12f}] {:s} {:s}'.format(i, g_hls, e_hls, 'E' if excited_state_hls else 'G', '***' if mismatch else ''))
    print('')
    if (mismatch):
        logit_errors = logit_errors + 1
    if (not excited_state_ref):
        class_errors = class_errors + 1

In [ ]:
logit_error_rate = (logit_errors * 100.) / test_size
class_error_rate = (class_errors * 100.) / test_size
print('Logit errors: {:d}/{:d} ({:.2f}%)'.format(logit_errors, test_size, logit_error_rate))
print('Class errors: {:d}/{:d} ({:.2f}%)'.format(class_errors, test_size, class_error_rate))

If you run with the model and **board** data on files, you should expect
```
Logit errors: 440/3100 (14.19%)
Class errors: 0/3100 (0.00%)
```
**ATTENTION:** We should expect _0_ errors because we are comparing board vs. hls implementations.

Given these results, my current guess is a problem with timing/triggering. The NN reads 200 I/Q samples after a trigger event + some delay. The delay in some cases should be a little earlier or later than expected.

```
I, Q, I, Q, I, Q, I, Q, I, Q, I, Q, I, Q, I, Q, I, Q, I, Q, I, Q, I, Q, I, Q, I, Q, I, Q, ..., I, Q, I, Q, I, Q
      ^
      Trigger
      <-------->
         delay
                  <--------------------------------- 200 samples -------------------------------------------->

             <=================================== 200 samples ==========================================>
```